## Running Backs

In [68]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
import pandas as pd
pd.options.display.max_columns = None # See all columns
import warnings
warnings.filterwarnings('ignore')
from fantasy_football import Fantasy

# Import fantasy class which will be used for building datasets
football = Fantasy()

##### First, lets grab our RB data from our data-collection notebook

In [170]:
%store -r RB_data

##### Next, we pull more advanced data for rushing and receiving

In [124]:
# Rushing advanced
rushing_advanced = football.getAdvancedStats("rushing", "2020")

In [125]:
# Receiving advanced
receiving_advanced = football.getAdvancedStats("receiving", "2020") 

In [97]:
receiving_advanced.head()

,Player,Tm,Age,YBC/R,YAC/R,ADOT,Rec/Br,Drop%,Int,Rat
0,S. Diggs,Buf,27,8.4,3.7,10.1,25.4,4.8,2,115.4
1,D. Adams,Gnb,28,6.8,5.2,8.9,57.5,0.7,3,136.0
2,D. Hopkins,Ari,28,7.6,4.6,8.9,7.7,1.3,2,105.9
3,D. Waller,Lvr,28,5.8,5.3,7.8,15.3,2.8,3,110.0
4,T. Kelce,Kan,31,7.9,5.6,8.5,10.5,1.4,3,119.8


In [169]:
rushing_advanced.head()

,Player,YBC/Att,YAC/Att,Att/Br
0,D. Henry,2.5,2.8,11.1
1,D. Cook,2.6,2.4,9.5
2,J. Jacobs,1.9,2.0,13.7
3,D. Montgomery,1.9,2.4,8.5
4,E. Elliott,1.9,2.1,16.3


##### Lets merge our advanced data

In [162]:
RB_advanced = pd.merge(RB_data, rushing_advanced, on='Player', how='left')
RB_advanced = pd.merge(RB_advanced, receiving_advanced, on='Player', how='left')

##### Some players did not match, but we still want to include them! Lets fill in the resulting NaN columns with column averages.

First, lets convert the relevant columns to float types

In [163]:
cols_to_change = ['YBC/Att', 'YAC/Att', 'Att/Br', 'Age', 'YBC/R', 'YAC/R', 'ADOT', 'Rec/Br', 'Drop%', 'Int', 'Rat']
RB_advanced[cols_to_change] = RB_advanced[cols_to_change].replace('', pd.np.nan)
for col in cols_to_change:
    RB_advanced[col] = RB_advanced[col].astype(float)


In [164]:
numeric_cols = RB_advanced.select_dtypes(include=['float64', 'int64']).columns
RB_advanced[numeric_cols] = RB_advanced[numeric_cols].fillna(RB_advanced[numeric_cols].mean())

##### Finally, lets make sure to drop any duplicates

In [165]:
RB_advanced = RB_advanced.drop_duplicates(subset='Player', keep='first')

##### Lets check out our merged dataset!

In [168]:
RB_advanced

,Player,Pos,Tm,GP,Att,Rushing_Yds,Rushing_Td,Rushing_1st,Tgt,Rec,Receiving_Yds,Receiving_Td,Receiving_1st,Return_Yds,Return_Td,2PT,Total,Lost,PPG,Rank,YBC/Att,YAC/Att,Att/Br,Age,YBC/R,YAC/R,ADOT,Rec/Br,Drop%,Int,Rat
0,N. Chubb,RB,Cle,12,15.833333,88.916667,1.000000,4.583333,1.500000,1.333333,12.500000,0.000000,0.500000,0.0,0.0,0.0,0.083333,0.083333,13.950000,25.0,2.900000,2.700000,9.000000,25.000000,1.100000,8.300000,2.700000,4.000000,0.0,0.000000,101.400000
1,A. Jones,RB,Gb,14,14.357143,78.857143,0.642857,3.714286,4.500000,3.357143,25.357143,0.142857,1.071429,0.0,0.0,0.0,0.142857,0.000000,11.800000,18.0,2.500000,2.900000,14.400000,26.000000,-0.200000,7.800000,1.200000,7.800000,6.3,0.000000,98.300000
2,J. Taylor,RB,Ind,15,15.466667,77.933333,0.733333,4.600000,2.600000,2.400000,19.933333,0.066667,0.800000,0.0,0.0,0.0,0.066667,0.066667,19.594118,3.0,3.000000,2.100000,15.500000,21.000000,-1.900000,10.200000,-1.300000,12.000000,2.6,0.000000,107.200000
6,S. Barkley,RB,Nyg,2,9.500000,17.000000,0.000000,0.500000,4.500000,3.000000,30.000000,0.000000,1.500000,0.0,0.0,0.0,0.000000,0.000000,8.276923,20.0,-0.100000,1.800000,19.000000,23.000000,0.800000,9.200000,3.800000,6.000000,11.1,0.000000,85.400000
7,A. Ekeler,RB,Lac,10,11.600000,53.000000,0.100000,2.600000,6.500000,5.400000,40.300000,0.200000,2.000000,0.0,0.0,0.0,0.100000,0.000000,17.112500,23.0,2.400000,2.200000,8.900000,25.000000,-1.300000,8.800000,-1.200000,7.700000,3.1,0.000000,102.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,T. Jones Jr.,RB,No,1,3.000000,13.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,1.554545,29.0,2.634783,1.810145,17.389815,25.132353,1.260294,6.823529,2.105147,12.801064,7.7,0.447761,93.603676
139,K. Vaughn,RB,Tb,10,2.600000,10.900000,0.000000,0.500000,1.000000,0.500000,3.400000,0.100000,0.200000,0.0,0.0,0.0,0.100000,0.100000,2.716667,12.0,2.400000,1.800000,13.000000,23.000000,2.200000,4.600000,2.400000,12.801064,30.0,0.000000,91.200000
140,D. Freeman,RB,Fa,5,10.800000,34.400000,0.200000,1.800000,2.000000,1.400000,11.600000,0.000000,0.600000,0.0,0.0,0.0,0.000000,0.000000,7.037500,16.0,1.900000,1.300000,27.000000,28.000000,-1.700000,10.000000,-1.400000,12.801064,20.0,0.000000,84.600000
141,M. Breida,RB,Buf,12,4.916667,21.166667,0.000000,0.916667,0.833333,0.750000,8.000000,0.000000,0.416667,0.0,0.0,0.0,0.166667,0.083333,3.966667,32.0,2.700000,1.600000,29.500000,25.000000,0.000000,10.700000,-0.100000,9.000000,0.0,0.000000,106.700000


### Next step, normalize values